In [417]:
import torch 

# Tensor simple operations

In [418]:
tensor2d = torch.tensor([[2, 3, 7], [3, 4, 8]])
tensor2d

tensor([[2, 3, 7],
        [3, 4, 8]])

In [419]:
tensor2d.shape

torch.Size([2, 3])

In [420]:
# reshape tensors
tensor2d_reshape = tensor2d.reshape(3, 2)
tensor2d_reshape

tensor([[2, 3],
        [7, 3],
        [4, 8]])

In [421]:
# Transposing
tensor2d.T

tensor([[2, 3],
        [3, 4],
        [7, 8]])

In [422]:
# Matrix Multiplication (2 methods)
tensor2d.matmul(tensor2d_reshape)
tensor2d @ tensor2d_reshape

tensor([[53, 71],
        [66, 85]])

# Models on computation graphs

## Simple Logistic Regression using Neural Nets

In [423]:
import torch.nn.functional as F
from torch.autograd import grad

### Without Backpropagation

In [424]:
true = torch.tensor([1.0], dtype=torch.float32) # true labels

# Network
inputs = torch.tensor([1.1], dtype=torch.float32) # input feature
weights = torch.tensor([2.2], dtype=torch.float32) # weight params
bias = torch.tensor([0.0], dtype=torch.float32) # bias unit
function_output = inputs * weights + bias
activation_output = torch.sigmoid(function_output)

# Check loss after output()
loss_output = F.binary_cross_entropy(activation_output, true)
loss_output

tensor(0.0852)

### With Backpropagation

In [425]:
true = torch.tensor([1.0], dtype=torch.float32) # true labels

# Network
inputs = torch.tensor([1.1], dtype=torch.float32) # input feature
weights = torch.tensor([2.2], dtype=torch.float32, requires_grad=True) # weight params
bias = torch.tensor([0.0], dtype=torch.float32, requires_grad=True) # bias unit
function_output = inputs * weights + bias
activation_output = torch.sigmoid(function_output)

# Check loss after output()
loss_output = F.binary_cross_entropy(activation_output, true)

# apply backpropagation using comparing true and activation output.
grad_L_weights = grad(loss_output, weights, retain_graph=True)
grad_L_bias = grad(loss_output, bias, retain_graph=True)

print(grad_L_weights)
print(grad_L_bias)
print(loss_output)

# or instaed of typing the above code to output the grad_L outputs, we can use the following
loss_output.backward()
print(weights.grad)
print(bias.grad)
print(loss_output)

(tensor([-0.0898]),)
(tensor([-0.0817]),)
tensor(0.0852, grad_fn=<BinaryCrossEntropyBackward0>)
tensor([-0.0898])
tensor([-0.0817])
tensor(0.0852, grad_fn=<BinaryCrossEntropyBackward0>)


### Apply forward propagation after using the new weight and bias params

In [426]:
true = torch.tensor([1.0], dtype=torch.float32) # true labels

# Network
inputs = torch.tensor([1.1], dtype=torch.float32) # input feature
weights = torch.tensor([-0.8462], dtype=torch.float32, requires_grad=True) # weight params
bias = torch.tensor([-0.7693], dtype=torch.float32, requires_grad=True) # bias unit
function_output = inputs * weights + bias
activation_output = torch.sigmoid(function_output)

# Check loss after output()
loss_output = F.binary_cross_entropy(activation_output, true)

# apply backpropagation using comparing true and activation output.
grad_L_weights = grad(loss_output, weights, retain_graph=True)
grad_L_bias = grad(loss_output, bias, retain_graph=True)

print(grad_L_weights)
print(grad_L_bias)
print(loss_output)

# or instaed of typing the above code to output the grad_L outputs, we can use the following
loss_output.backward()
print(weights.grad)
print(bias.grad)
print(loss_output)

(tensor([-0.9301]),)
(tensor([-0.8456]),)
tensor(1.8679, grad_fn=<BinaryCrossEntropyBackward0>)
tensor([-0.9301])
tensor([-0.8456])
tensor(1.8679, grad_fn=<BinaryCrossEntropyBackward0>)


# Multilayer perceptron with 2 hidden layers

In [427]:
# Inheriting the Module class
class NeuralNetwork(torch.nn.Module):
    def __init__(self, num_inputs, num_outputs):
        # super init required due to inheritance. 
        super().__init__()
    
        self.layers = torch.nn.Sequential(
            
            # Input Layer
            torch.nn.Linear(num_inputs, 30, bias=True), # (inputs, outputs)
            torch.nn.ReLU(),

            # 2nd Hidden layer
            torch.nn.Linear(30, 20), # input should match the previous layers's outputs
            torch.nn.ReLU(),

            # Output layer
            torch.nn.Linear(20, num_outputs),
        )
    
    def forward(self, x):
        logits = self.layers(x)
        return logits

In [428]:
model = NeuralNetwork(50, 3)
print(model)

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=50, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
  )
)


## Check total number of trainable parameters in the model

In [429]:
# This includes both the weights and the bias parameters
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total number of trainable model parameters: {num_params}")

Total number of trainable model parameters: 2213


In [430]:
print(model.layers[0].weight.shape)

torch.Size([30, 50])


In [431]:
# To see reproducible results (instead of having initalise random weights, we can use a random sesed to map to a pre determinded set of random numbers).
torch.manual_seed(123)
X = torch.rand((1, 50)) # should match the size of the input
model = NeuralNetwork(50, 3)
print(model.layers[0].weight)

Parameter containing:
tensor([[ 0.0502,  0.0307,  0.0333,  ...,  0.0951,  0.1134, -0.0297],
        [ 0.1077, -0.1108,  0.0122,  ...,  0.0108, -0.1049, -0.1063],
        [-0.0920, -0.0480,  0.0105,  ..., -0.0923,  0.1201,  0.0330],
        ...,
        [ 0.1359,  0.0175, -0.0673,  ...,  0.0674,  0.0676,  0.1058],
        [ 0.0790,  0.1343, -0.0293,  ...,  0.0344, -0.0971, -0.0509],
        [-0.1250,  0.0513,  0.0366,  ..., -0.1370,  0.1074, -0.0704]],
       requires_grad=True)


In [432]:
# Logit values should add up to one. 
with torch.no_grad():
    output = torch.softmax(model(X), dim=1)
print(output)

tensor([[0.2801, 0.3635, 0.3565]])


# Creating Datasets and Dataloader

## Dataset Toy

In [433]:
# 2 features input (binary classification)

X_train = torch.tensor([
    [-1.2, 3.1],
    [-0.9, 2.9],
    [-0.5, 2.6],
    [2.3, -1.1],
    [2.7, -1.5]
])

y_train = torch.tensor([0, 0, 0, 1, 1])

X_test = torch.tensor([
    [-0.8, 2.8],
    [2.6, -1.6],
])

y_test = torch.tensor([0, 1])

In [434]:
from torch.utils.data import Dataset

class ToyDataset(Dataset):
    def __init__(self, X, y):
        self.features = X
        self.labels = y
    
    def __getitem__(self, index):
        one_x = self.features[index]
        one_y = self.labels[index]
        return one_x, one_y
    
    def __len__(self):
        return self.labels.shape[0]

In [435]:
train_ds = ToyDataset(X_train, y_train)
test_ds = ToyDataset(X_test, y_test)
print(len(train_ds))

5


## Dataloader

In [436]:
from torch.utils.data import DataLoader

torch.manual_seed(123)
train_loader = DataLoader(
    dataset=train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=0,
    drop_last=True
)

test_loader = DataLoader(
    dataset=test_ds,
    batch_size=2,
    shuffle=False,
    num_workers=0
)

In [437]:
for idx, (x, y) in enumerate(train_loader):
    print(f"Batch {idx+1}: X: {x}, y: {y}")

Batch 1: X: tensor([[ 2.3000, -1.1000],
        [-0.9000,  2.9000]]), y: tensor([1, 0])
Batch 2: X: tensor([[-1.2000,  3.1000],
        [-0.5000,  2.6000]]), y: tensor([0, 0])


# Training loop

In [ ]:
import torch.nn.functional as F
model = NeuralNetwork(num_inputs=2, num_outputs=2)

# optimiser
optimizer=torch.optim.SGD(model.parameters(), lr=0.001)

# epoch count
num_epochs=3
for epoch in range(num_epochs):
    model.train()
    for batch_idx, (features, labels) in enumerate(train_loader):
        logits = model(features)
        loss = F.cross_entropy(logits, labels)
        optimizer.zero_grad()
        optimizer.step()

        ## logging purposes
        print(
            f"Epoch: {epoch+1:02d}/{num_epochs:02d}"
            f" | Batch {batch_idx+1:02d}/{len(train_loader):02d}"
            f" | Train Loss: {loss:.2f}"
        )

Epoch: 01/03 | Batch 01/02 | Train Loss: 0.75
Epoch: 01/03 | Batch 02/02 | Train Loss: 0.67
Epoch: 02/03 | Batch 01/02 | Train Loss: 0.59
Epoch: 02/03 | Batch 02/02 | Train Loss: 0.75
Epoch: 03/03 | Batch 01/02 | Train Loss: 0.66
Epoch: 03/03 | Batch 02/02 | Train Loss: 0.75
